In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt


In [2]:
!pip install openpyxl -q


In [3]:
df = pd.read_excel('Game_ratings_PS4.xlsx')

In [4]:
df.head()

,Unnamed: 0,Users,Game_name,user_rating
0,0,/user/OVLange,Red Dead Redemption 2,10
1,1,/user/MisterLupus,Red Dead Redemption 2,7
2,2,/user/thehoneyman,Red Dead Redemption 2,7
3,3,/user/Yozakura,Red Dead Redemption 2,6
4,4,/user/colinsmall34,Red Dead Redemption 2,7


In [5]:
df.drop('Unnamed: 0',axis=1)


,Users,Game_name,user_rating
0,/user/OVLange,Red Dead Redemption 2,10
1,/user/MisterLupus,Red Dead Redemption 2,7
2,/user/thehoneyman,Red Dead Redemption 2,7
3,/user/Yozakura,Red Dead Redemption 2,6
4,/user/colinsmall34,Red Dead Redemption 2,7
...,...,...,...
258558,/user/DarCbrGr,Tony Hawk's Pro Skater 5,0
258559,/user/Norm_HD,Tony Hawk's Pro Skater 5,0
258560,/user/Reeeeeeeeee,Tony Hawk's Pro Skater 5,1
258561,/user/13Impossible,Tony Hawk's Pro Skater 5,2


In [6]:
df['Game_name'].value_counts()

The Last of Us Part II    78611
Ghost of Tsushima          9300
Cyberpunk 2077             5238
God of War                 4967
Red Dead Redemption 2      4549
                          ...  
Mitsurugi Kamui Hikae         1
RiftStar Raiders              1
Vacation Simulator            1
King of Seas                  1
Traffic Jams                  1
Name: Game_name, Length: 2247, dtype: int64

In [7]:
df1 = df['Users'].value_counts().rename_axis('Users').reset_index(name='counts')
df1 = df1[df1['counts'] > 4]
df_games = df.merge(df1,how='inner', on ='Users') 

In [8]:
df_games = df_games.drop('Unnamed: 0', 1)
df_games.head()

,Users,Game_name,user_rating,counts
0,/user/colinsmall34,Red Dead Redemption 2,7,19
1,/user/colinsmall34,Titanfall 2,9,19
2,/user/colinsmall34,Battlefield 1,9,19
3,/user/colinsmall34,Rise of the Tomb Raider: 20 Year Celebration,8,19
4,/user/colinsmall34,DOOM,8,19


In [9]:
matrix = df_games.pivot_table(columns='Game_name', index='Users', values='user_rating', fill_value=0)
matrix

Game_name,10 Second Ninja X,1001 Spikes,100ft Robot Golf,11-11: Memories Retold,13 Sentinels: Aegis Rim,1971 Project Helios,1979 Revolution: Black Friday,2064: Read Only Memories,20XX,2Dark,...,Zone of the Enders: The 2nd Runner MARS,Zotrix,de Blob 2,eFootball PES 2020,eFootball PES 2021,flOw,flower,inFamous: First Light,inFamous: Second Son,theHunter: Call of the Wild
Users,,,,,,,,,,,,,,,,,,,,,
/user/02450ephiz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
/user/1080pony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
/user/1111me,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
/user/1234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7,0
/user/1234marcos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/user/ziGihouse,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
/user/zisisaxl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
/user/zkg03mjmk,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def center(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row
matrix_std = matrix.apply(center)

In [11]:
matrix_std

Game_name,10 Second Ninja X,1001 Spikes,100ft Robot Golf,11-11: Memories Retold,13 Sentinels: Aegis Rim,1971 Project Helios,1979 Revolution: Black Friday,2064: Read Only Memories,20XX,2Dark,...,Zone of the Enders: The 2nd Runner MARS,Zotrix,de Blob 2,eFootball PES 2020,eFootball PES 2021,flOw,flower,inFamous: First Light,inFamous: Second Son,theHunter: Call of the Wild
Users,,,,,,,,,,,,,,,,,,,,,
/user/02450ephiz,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,-0.05394,-0.001222
/user/1080pony,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,-0.05394,-0.001222
/user/1111me,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,-0.05394,-0.001222
/user/1234,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,0.64606,-0.001222
/user/1234marcos,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,-0.05394,-0.001222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/user/ziGihouse,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,-0.05394,-0.001222
/user/zisisaxl,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,-0.05394,-0.001222
/user/zkg03mjmk,-0.000456,-0.000433,-0.000219,-0.000588,-0.005381,-0.000182,-0.000182,-0.001003,-0.000274,-0.000388,...,-0.000689,-0.000319,-0.000284,-0.007862,-0.002244,-0.000967,-0.006457,-0.012714,-0.05394,-0.001222


In [12]:
def gameRec(g):
    dota = matrix_std[g]
#Calculate Pearson Sim with all other games.
    dota = matrix.corrwith(dota).dropna()
#create a DF to show how many times each game has been played and the mean time it has been played
    gameData = df_games.groupby('Game_name').agg({'user_rating': [np.size, np.mean]})
#Filter out any game played by less than n players.
    gameSim = gameData['user_rating']['size'] >= 0
    df = gameData[gameSim].join(pd.DataFrame(dota, columns=['similarity']))
    df = df.drop(g,axis=0)
    return df.sort_values(['similarity'], ascending=False)[:20]

In [13]:
gameRec('Call of Duty: Modern Warfare')

/Users/sameermahajan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(user_rating, size)","(user_rating, mean)",similarity
Game_name,,,
Call of Duty: Warzone,50,5.000000,0.188313
Call of Duty: Black Ops Cold War,75,3.813333,0.125690
Call of Duty: Black Ops 4,113,3.619469,0.114070
Call of Duty: Modern Warfare 2 Campaign Remastered,32,7.062500,0.109102
Sword Art Online: Alicization Lycoris,6,6.166667,0.108090
FIFA 20,146,2.164384,0.106987
Call of Duty: Modern Warfare Remastered,72,6.930556,0.102941
Life is Strange 2: Episode 1 - Roads,4,4.250000,0.102311
Senran Kagura: Peach Beach Splash,8,7.500000,0.102004


In [14]:
def gameRec_modified(g):
    dota = matrix_std[g]
#Calculate Pearson Sim with all other games.
    dota = matrix.corrwith(dota).dropna()
#create a DF to show how many times each game has been played and the mean time it has been played
    gameData = df_games.groupby('Game_name').agg({'user_rating': [np.size, np.mean]})
#Filter out any game played by less than n players.
    gameSim = gameData['user_rating']['size'] >= 0
    df = gameData[gameSim].join(pd.DataFrame(dota, columns=['similarity']))
    return df['similarity']

In [15]:
#multiple games recommendation
x = list(matrix.keys())
df_4 = pd.DataFrame(x , columns= ["Game_name"])

def multi_game(game_list):
    df_4['Total_similarity'] = 0
    for i in game_list:
        new_col = list(gameRec_modified(i))
        df_4[i] = new_col
        df_4['Total_similarity'] = df_4['Total_similarity'] + df_4[i]
        
    df_4['Total_similarity'] = df_4['Total_similarity']/len(game_list)
    
    return df_4[['Game_name','Total_similarity']].sort_values(['Total_similarity'], ascending=False)[:20]

In [16]:
multi_game(['inFamous: Second Son','FIFA 15',"Far Cry 4"])

/Users/sameermahajan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/sameermahajan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/sameermahajan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,Game_name,Total_similarity
557,Far Cry 4,0.412767
2032,inFamous: Second Son,0.408055
533,FIFA 15,0.375907
1946,Watch Dogs,0.201115
2031,inFamous: First Light,0.159075
223,Call of Duty: Advanced Warfare,0.150261
969,Middle-earth: Shadow of Mordor,0.149869
816,Killzone: Shadow Fall,0.145163
892,LittleBigPlanet 3,0.129813
397,Destiny,0.128376


In [86]:
#multiple games recommendation
x = list(matrix.keys())
df_4 = pd.DataFrame(x , columns= ["Game_name"])

def multi_game(game_dict):
    df_4['Total_similarity'] = 0
    for key in game_dict:
        new_col = list(gameRec_modified(key))
        df_4[key] = pd.DataFrame(new_col)
        df_4[key] = df_4[key]*(0.1)*game_dict.get(key)

    #     df_4[key] = game_dict.get(key)*new_col
    #     # df_4[key].head()
        df_4['Total_similarity'] = df_4['Total_similarity'] + df_4[key]

    df_4['Total_similarity'] = df_4['Total_similarity']/len(game_dict)
    
    return df_4[['Game_name','Total_similarity']].sort_values(['Total_similarity'], ascending=False)[:20]

In [92]:
multi_game({'FIFA 15':3,"Far Cry 4":10,'FIFA 20':5})

/Users/sameermahajan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/sameermahajan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/sameermahajan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,Game_name,Total_similarity
557,Far Cry 4,0.338505
538,FIFA 20,0.166987
533,FIFA 15,0.130581
1946,Watch Dogs,0.089937
969,Middle-earth: Shadow of Mordor,0.086176
480,Dying Light,0.069340
223,Call of Duty: Advanced Warfare,0.069173
666,Grand Theft Auto V,0.068577
537,FIFA 19,0.065692
892,LittleBigPlanet 3,0.063355
